In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time, math, os
import re
from datetime import datetime

# 爬取團購貼文link

In [156]:
def crawlLinklist(email,password):    
   #跳過chrome通知允許alert windows
    driverpath="/Users/benbilly3/Desktop/創潮爬蟲/chromedriver"#74版，win的省略
    options = webdriver.ChromeOptions()
    prefs = {'profile.default_content_setting_values' :{'notifications' : 2}}
    options.add_experimental_option('prefs',prefs)

    #操控chromedriver
    browser=webdriver.Chrome(executable_path=driverpath,options = options)#模擬瀏覽器，跳過通知允許
    url='https://www.facebook.com/'
    browser.get(url)
    search=browser.find_element_by_id('email')
    search.send_keys(email)#填mail
    time.sleep(3)
    search=browser.find_element_by_id('pass')
    search.send_keys(password)#填密碼
    search.send_keys(Keys.RETURN) # hit return after you enter search text,使用Keys物件功能
    browser.get('https://www.facebook.com/groups/TDa.BuyWine/')
    #滾動
    jsCode = "var q=document.documentElement.scrollTop=100000"
    for i in range(50):
        browser.execute_script(jsCode)
        time.sleep(1)

    #get_attribute取出貼文超連結
    link_list=[link.get_attribute('href') for link in browser.find_elements_by_xpath('//a[@class="_5pcq"]') ]
    browser.close()
    return link_list

# crawlLinklist('benbilly3@gmail.com','benbilly3')


# 爬取留言及購買資訊

### 抓取 po 文內文

```
tag=browser.find_elements_by_tag_name('p')
d=[]
for i in range(len(tag)):
    d.append(tag[i].text)
print(d)
```


### data-* list:

- comments 根元素: data-testid='UFI2CommentsList/root_depth_0'
- 讀取更多 comments 按鈕: data-testid='UFI2CommentsPagerRenderer/pager_depth_0'

### 使用Xpath 抓取 HTML5 [data-*]

```
loadCommentButton = browser.find_element_by_xpath("//a[@data-testid='UFI2CommentsPagerRenderer/pager_depth_0']")
```

### 連續搜尋

```
driver.find_element_by_xpath("//elementType[@firstAttributeTypte = 'firstAttributeValue'][@secondAttributeType='secondAttributeValue'][@thirdAttributeType='thirdAttributeValue']....");
```

In [6]:
def InnoproFBCrawler(email,password,orderUrl):
    driverpath="/Users/benbilly3/Desktop/Innopro-master/chromedriver"#74版，win的省略
    options = webdriver.ChromeOptions()
    prefs = {'profile.default_content_setting_values' :{'notifications' : 2}}
    options.add_experimental_option('prefs',prefs)

    #操控chromedriver
    browser=webdriver.Chrome(executable_path=driverpath,options = options)#模擬瀏覽器，跳過通知允許
    url='https://www.facebook.com/'
    browser.get(url)
    search=browser.find_element_by_id('email')
    search.send_keys(email)#填mail
    time.sleep(3)
    search=browser.find_element_by_id('pass')
    search.send_keys(password)#填密碼
    search.send_keys(Keys.RETURN) # hit return after you enter search text,使用Keys物件功能
    browser.get(orderUrl)#進入商品貼文網址
    comments = pd.DataFrame(columns=['username','orderCount','fbURL'])
    pd.set_option('max_colwidth',200)

    comments = pd.DataFrame(
        columns=['comment_order', 'username', 'fbuid', 'fbURL', 'commentTime', 'orderCount', 'commentContent'])

    try:
        loadCommentButton = browser.find_element_by_xpath(
            '//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'
        )
        print('comment counts more than 50')
        loadCommentOffsetY = loadCommentButton.location['y'] - 300
        browser.execute_script(
            "window.scrollTo(0, {});".format(loadCommentOffsetY))

    except NoSuchElementException:
        print('comment counts less than 50')
        print('all comments loaded')
        isAllCommentsLoaded = True
    except Exception as e:
        print(e.msg)
    else:
        isAllCommentsLoaded = False

    while isAllCommentsLoaded == False:
        try:
            loadCommentButton.click()
            time.sleep(0.5)
        except StaleElementReferenceException:
            print('all comments loaded')
            isAllCommentsLoaded = True
        except Exception as e:
            print(e.msg)

    # 抓取留言資料
    comments_list = browser.find_element_by_xpath(
        ".//div[@data-testid='UFI2CommentsList/root_depth_0']")
    comment_roots = comments_list.find_elements_by_xpath(
        ".//div[@data-testid='UFI2Comment/root_depth_0']")

    # FB 留言時間字串格式
    # https://docs.python.org/zh-cn/3/library/datetime.html#strftime-and-strptime-behavior
    datetime_format = "%Y, %B %d, %A at %p %I:%M "
    for index, comment_root in enumerate(comment_roots):
        comment_body = comment_root.find_element_by_xpath(
            ".//div[@data-testid='UFI2Comment/body']")
        username = comment_body.find_element_by_xpath(
            ".//a[contains(@data-hovercard,'')]").text
        
        profile_pic_a = comment_root.find_element_by_xpath("div[contains(@class,'lfloat')]//a[contains(@data-hovercard,'user.php?id=')]")
        id_link = profile_pic_a.get_attribute('data-hovercard')
        fbuid = id_link[id_link.find('user.php?id=')+12:]
        comment_content = comment_body.text.split(' ')[-1]
        order_count = comment_content.replace('+', '').replace('＋', '')
        fbUrl = comment_root.find_element_by_xpath(
            ".//a[contains(@href,'/')]").get_attribute('href')

        comment_time = comment_root.find_element_by_tag_name(
            'abbr').get_attribute('data-tooltip-content')
        comment_time = datetime.strptime(comment_time, datetime_format)
        comments.loc[index] = (
            index, username, fbuid, fbUrl, comment_time, order_count, comment_content)

    post_and_comments_html = browser.find_element_by_xpath(
        "//div[@data-testid='newsFeedStream']").get_attribute('outerHTML')
    postDatas.at[postDatas['link'] == postUrl, 'html'] = post_and_comments_html

    # 有兩種形式
    if browser.find_elements_by_class_name('_3w8y') != []:
        tag = browser.find_elements_by_class_name('_3w8y')
    else:
        tag = browser.find_elements_by_tag_name('p')

    wine_main = [tag[i].text for i in range(len(tag))]  # 抓取商品內容主文
    # 抓<p>標籤第一行的"wineID"
    index_data = wine_main[0]
    wine_id = index_data[index_data.find("-")+1:index_data.find(">")]
    comments["wineID"] = wine_id

    # 抓<p>標籤第一行的"publishDate(團購發布年月)"、"crawlDate(爬取時間)"
    year = index_data[1:5]
    month = index_data[index_data.find("/")+1:index_data.find("月")]
    publish_date = datetime.strptime(year+'-'+month, '%Y-%m')
    crawl_day = datetime.now().strftime('%Y-%m-%d %H:%M')

    comments["publishDate"] = publish_date
    comments["crawlDate"] = crawl_day

    # 過濾掉非數字留言
    comments["orderCount"] = comments["orderCount"].apply(
        lambda s: pd.to_numeric(s, errors='coerce'))
    df = comments.dropna()
    df = df.groupby('fbURL', as_index=False).max()
    return df, year, month, wine_id

#test
InnoproFBCrawler("benbilly3@gmail.com","benbilly3@",'https://www.facebook.com/groups/1993881810669370/permalink/2552092671514945/')


NameError: name 'NoSuchElementException' is not defined

# 迴圈取資料

In [161]:
def WineSpider(email,password,start=None,end=None):
    link_list=crawlLinklist(email,password)
    for orderUrl in link_list[start:end]:
        try:
            InnoproFBCrawler(email,password,orderUrl)
            print("sucess",orderUrl )
        #有一些貼文是雜務，不是賣酒貼文，如宅配通知或哈拉區
        except:
            print("error",orderUrl )
            pass
        time.sleep(15)
 

In [162]:
WineSpider("benbilly3@gmail.com","benbilly3",5,30)

973
sucess https://www.facebook.com/groups/TDa.BuyWine/permalink/2406451966299262/
3806
sucess https://www.facebook.com/groups/TDa.BuyWine/permalink/2406446529633139/
3803
sucess https://www.facebook.com/groups/TDa.BuyWine/permalink/2406445946299864/
3808
sucess https://www.facebook.com/groups/TDa.BuyWine/permalink/2406443809633411/
3805
sucess https://www.facebook.com/groups/TDa.BuyWine/permalink/2406445232966602/
4264
sucess https://www.facebook.com/groups/TDa.BuyWine/permalink/2406447042966421/
3796
sucess https://www.facebook.com/groups/TDa.BuyWine/permalink/2406444976299961/
3807
sucess https://www.facebook.com/groups/TDa.BuyWine/permalink/2406444136300045/
4265
sucess https://www.facebook.com/groups/TDa.BuyWine/permalink/2406450366299422/
2876
sucess https://www.facebook.com/groups/TDa.BuyWine/permalink/2406396439638148/
4391
sucess https://www.facebook.com/groups/TDa.BuyWine/permalink/2406414459636346/
2905
sucess https://www.facebook.com/groups/TDa.BuyWine/permalink/24064258229